In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import statistics as st

%matplotlib inline  

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
api_key = '00ef9804fcde0edd93b1b4821ee2f06a'
# api_key = 'c81352430e2fe3c941faf0814227562b'

data

In [4]:

''' Колонки '''
ticker = 'F'
# ticker = 'AAPL'


''' время '''
stime='1990-01-01'
ftime='2023-02-01'
period = '1mo'  # '1d' '1wk'

In [5]:
def get_data_from_ticker(tick, stime, ftime, interval=period):
    ticker = yf.Ticker(tick)

    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z

df = get_data_from_ticker(ticker, stime, None, period)
df.tail(3)

,F
Date,
2022-11-01 00:00:00-04:00,13.756207
2022-12-01 00:00:00-05:00,11.630000
2023-01-01 00:00:00-05:00,12.400000


change index

In [6]:
def change_index(df):
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        t = df_index[i].date()
        df_index[i] = t.strftime('%Y-%m-%d')
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

df = change_index(df)
df.head(3)

,F
Date_index,
1990-01-01,2.560526
1990-02-01,2.778589
1990-03-01,2.854094


In [7]:
# df.loc['1999-01-01']

In [8]:
# df_st_gr = fa.financial_statement_growth(ticker, api_key, period='annual') #quarter 

In [9]:
# df_st_gr

In [10]:
key_metrics = fa.key_metrics(ticker, api_key, period='annual')
df_key_metr = key_metrics.T
df_key_metr.head(3)

,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
2021,FY,34.162115,4.494362,3.95565,2.39539,12.426209,12.157103,12,12.157103,36.717615,77984143591.899994,196929143591.899994,4.347669,0.571979,4.93977,8.157337,1.607291,1.607291,1.444387,6.38924,12.474133,20.599283,0.230008,0.122589,1.843566,0.810835,3.859094,1.201362,0.641106,0.880136,0.005168,0.022468,0,0,0.00284,-0.394438,-0.045672,-1.044799,0.002237,35.062296,0.027197,0.069983,-30.030757,18269000000,47892000000,-99417000000,2.874853,48153500000,22276500000,11436500000,117.559978,71.149707,38.409826,3.104798,5.130028,9.502777,0.36969,-1.560261
2020,FY,32.002014,-0.321923,6.108482,4.663227,12.575132,7.724641,7.653914,7.724641,43.607853,41835688808.099998,179590688808.100006,-32.709686,0.329042,1.723832,2.258093,1.36317,1.36317,1.412498,10.822628,7.400004,9.693458,-0.030572,0.442852,3.627631,0.884716,8.301495,1.201169,-0.429797,-18.97498,0.014246,-0.465989,0,0,0.001504,-0.236598,-0.045161,-0.770015,0.001565,7.480083,-0.025919,-0.004793,-35.688271,19552000000,30409000000,-119706000000,5.311111,52394000000,22204000000,10808000000,150.410637,71.878636,34.987583,2.42669,5.078004,10.432272,-0.041675,-1.445255
2019,FY,39.249748,0.011833,4.440836,2.519386,8.723817,8.354733,8.248741,8.354733,42.105237,35112480794.400002,174329480794.399994,747.074059,0.225224,1.990616,3.508792,1.058083,1.058083,1.118213,9.508535,9.883184,17.420754,0.001339,0.284998,3.085973,0.871469,7.593378,1.16218,0.054558,375.297872,0.068038,50.829787,0,0,0.001802,-0.432678,-0.048954,-0.89894,0.001462,1.491427,0.003578,0.000182,-34.767623,15915000000,32764000000,-111260000000,4.722646,57641000000,21438500000,10797000000,147.236177,56.021063,29.228616,2.47901,6.515407,12.487762,0.001416,-1.92145


In [11]:
df_key_metr.index[0]

'2021'

In [12]:
df_key_metr = df_key_metr.loc['2022':'2000'].copy(deep=True)
index_years = df_key_metr.index.to_list()
index_years


['2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000']

In [13]:
# financial_ratios = fa.financial_ratios(ticker, api_key, period='annual')
# financial_ratios = financial_ratios.T
# financial_ratios

In [14]:
# df_fin_rat = financial_ratios
# df_fin_rat = df_fin_rat.loc['2022':'2000'].copy(deep=True)
# index_years = df_fin_rat.index.to_list()
# index_years

stock data detailed

In [15]:
# stock_data = fa.stock_data(ticker, period="max", interval="1mo")
# stock_data

In [16]:
# stock_data_det = fa.stock_data_detailed(ticker, api_key)
# stock_data_det

In [17]:
def get_price_from_df(df, list_dates):
    new_list = []
    for i in range(len(list_dates)):
        x = int(list_dates[i])
        x += 1
        x = str(x)
        r = df.loc[f'{x}-01-01']
        r = float(r)
        new_list.append(r)
        
    return new_list
        
stock_year_prices = get_price_from_df(df, index_years)
stock_year_prices

[19.659067153930664,
 10.146446228027344,
 8.356614112854004,
 7.818579196929932,
 9.092910766601562,
 9.701883316040039,
 8.761109352111816,
 10.372193336486816,
 10.218565940856934,
 8.616663932800293,
 8.112526893615723,
 10.418264389038086,
 7.080500602722168,
 1.2214518785476685,
 4.337133884429932,
 5.310375213623047,
 5.422718524932861,
 7.998786926269531,
 8.596181869506836,
 5.18708610534668,
 8.430548667907715,
 14.891738891601562]

In [18]:
df_key_metr[ticker] = list(stock_year_prices)

In [19]:
df_key_metr.drop('period', axis=1, inplace=True)

In [20]:
df_key_metr.tail(3)

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare,F
2002,89.840572,-0.538758,10.24354,6.242441,16.778999,3.073117,2.552501,3.073117,89.181968,16389190363.800001,166361190363.799988,-16.723664,0.100289,0.879579,1.443346,2.931877,2.931877,1.017998,11.473977,8.928309,14.650919,-0.059796,0.692835,29.020036,0.961086,10.34361,1.936011,None,-19.013265,0.045335,-0.758163,0,0,0.022868,-0.390597,-0.044536,-0.479541,0,6.10348,0.05215,-0.003466,-93.328615,93924000000,4643000000,-83828000000,29.020036,75744500000,20592500000,8894000000,221.32955,55.363841,20.359286,1.649125,6.592751,17.927937,-0.175313,-4.0011,5.187086
2001,89.237363,-2.996154,12.507692,7.881319,9.981868,4.647253,4.647253,4.647253,119.987912,27063399818,238223399818,-4.963029,0.166634,1.188868,1.88674,3.19974,3.19974,1.466784,19.613321,10.464918,16.607878,-0.20149,0.530015,19.748759,0.969415,17.385147,0.81399,None,-4.174583,0.071277,-0.35375,0,0,0,-0.369882,-0.051843,-0.426329,0,17.699926,-0.032621,-0.019718,-134.317857,-8286000000,None,-231825000000,25.819106,27773000000,18940500000,8499500000,7.083713,42.419899,16.752031,51.52665,8.604452,21.788403,-0.644715,-4.626374,8.430549
2000,114.675657,2.337829,22.767363,15.717465,12.115307,12.548887,12.548887,12.548887,147.002697,41968898813.599998,253739898813.600006,12.105249,0.246783,1.243007,1.800545,2.25518,2.25518,1.492026,11.325652,7.515102,10.88592,0.082609,0.555387,8.881193,0.934569,9.452375,0.808449,10.942155,9.738679,0.065549,0.793481,0,0,0,-0.309649,-0.061477,-0.704088,0,25.692136,0.064178,0.01219,-161.418071,-9314000000,None,-226501000000,11.640086,29333000000,18639500000,9161000000,13.461285,40.566319,20.219922,27.114796,8.997612,18.051504,0.186298,-7.049899,14.891739


In [21]:
def change_percent_all_columns(df):
    x = df.columns.to_list()
    for i in range(len(x)):
        v = df[x[i]].to_list()
        l = []
        for y in range(len(v)):
            try:
                # if y != len(v):
                #     vv = (v[y] / (v[y+1] / 100)) - 100
                #     l.append(vv)
                #     # print(vv)
                if y != len(v):
                    if v[y] < 0 and v[y+1] > 0:
                        l.append('Minus')
                    elif v[y] > 0 and v[y+1] < 0:
                        l.append('Plus')
                    else:
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv)  
                else:
                    vv = 0
                    l.append(vv)
                    # print(vv)
            except:
                l.append(0)
        df[f'change % {x[i]}'] = l
    return df

df_with_changes = change_percent_all_columns(df_key_metr)

            

In [22]:
x = df_with_changes[['shareholdersEquityPerShare', 'change % shareholdersEquityPerShare']]
x

,shareholdersEquityPerShare,change % shareholdersEquityPerShare
2021,12.157103,57.380815
2020,7.724641,-7.541735
2019,8.354733,-8.069024
2018,9.08805,3.672659
2017,8.766101,19.395678
2016,7.342059,1.740911
2015,7.216427,13.810376
2014,6.340746,-5.428355
2013,6.704701,60.396537
2012,4.180079,5.503316


In [23]:
# df_with_changes.columns.to_list()

In [24]:
df_with_changes = df_with_changes.reindex(index=df_with_changes.index[::-1])
# df_with_changes.head(3)

df_with_changes

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare,F,change % revenuePerShare,change % netIncomePerShare,change % operatingCashFlowPerShare,change % freeCashFlowPerShare,change % cashPerShare,change % bookValuePerShare,change % tangibleBookValuePerShare,change % shareholdersEquityPerShare,change % interestDebtPerShare,change % marketCap,change % enterpriseValue,change % peRatio,change % priceToSalesRatio,change % pocfratio,change % pfcfRatio,change % pbRatio,change % ptbRatio,change % evToSales,change % enterpriseValueOverEBITDA,change % evToOperatingCashFlow,change % evToFreeCashFlow,change % earningsYield,change % freeCashFlowYield,change % debtToEquity,change % debtToAssets,change % netDebtToEBITDA,change % currentRatio,change % interestCoverage,change % incomeQuality,change % dividendYield,change % payoutRatio,change % salesGeneralAndAdministrativeToRevenue,change % researchAndDdevelopementToRevenue,change % intangiblesToTotalAssets,change % capexToOperatingCashFlow,change % capexToRevenue,change % capexToDepreciation,change % stockBasedCompensationToRevenue,change % grahamNumber,change % roic,change % returnOnTangibleAssets,change % grahamNetNet,change % workingCapital,change % tangibleAssetValue,change % netCurrentAssetValue,change % investedCapital,change % averageReceivables,change % averagePayables,change % averageInventory,change % daysSalesOutstanding,change % daysPayablesOutstanding,change % daysOfInventoryOnHand,change % receivablesTurnover,change % payablesTurnover,change % inventoryTurnover,change % roe,change % capexPerShare,change % F
2000,114.675657,2.337829,22.767363,15.717465,12.115307,12.548887,12.548887,12.548887,147.002697,41968898813.599998,253739898813.600006,12.105249,0.246783,1.243007,1.800545,2.25518,2.25518,1.492026,11.325652,7.515102,10.88592,0.082609,0.555387,8.881193,0.934569,9.452375,0.808449,10.942155,9.738679,0.065549,0.793481,0,0,0,-0.309649,-0.061477,-0.704088,0,25.692136,0.064178,0.01219,-161.418071,-9314000000,None,-226501000000,11.640086,29333000000,18639500000,9161000000,13.461285,40.566319,20.219922,27.114796,8.997612,18.051504,0.186298,-7.049899,14.891739,0.000000,0,0,0,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0.000000,0,0,0,0,0.000000,0.000000,0,0,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,0,0.0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000
2001,89.237363,-2.996154,12.507692,7.881319,9.981868,4.647253,4.647253,4.647253,119.987912,27063399818,238223399818,-4.963029,0.166634,1.188868,1.88674,3.19974,3.19974,1.466784,19.613321,10.464918,16.607878,-0.20149,0.530015,19.748759,0.969415,17.385147,0.81399,None,-4.174583,0.071277,-0.35375,0,0,0,-0.369882,-0.051843,-0.426329,0,17.699926,-0.032621,-0.019718,-134.317857,-8286000000,None,-231825000000,25.819106,27773000000,18940500000,8499500000,7.083713,42.419899,16.752031,51.52665,8.604452,21.788403,-0.644715,-4.626374,8.430549,-22.182820,Minus,-45.063062,-49.856298,-17.609448

In [25]:
''' only plus '''
RevenuePerShare, operatingCashFlowPerShare

NameError: name 'RevenuePerShare' is not defined

In [ ]:
''' plus and minus '''
netIncomePerShare, 


Current tests

In [ ]:
df_some_columns = df_with_changes[[f'change % {ticker}', 'change % debtToAssets']]
df_some_columns.head(3)

In [ ]:
df_some_columns

In [ ]:
st.mean(df_some_columns[f'change % {ticker}'].to_list()[1:])

In [ ]:
df_with_changes[[ticker,f'change % {ticker}']]

Vizualization

In [ ]:
# Set the title and axis labels
plt.title('Percent')
plt.xlabel('Date')
plt.ylabel('Percent')

x = 'change % revenuePerShare'
y = 'change % cashPerShare'


# Plot close price and moving averages
plt.plot(df_with_changes[f'change % {ticker}'],lw=1, label=ticker)
plt.plot(df_with_changes[x],'g',lw=1, label=x)
plt.plot(df_with_changes[y],'r', lw=1, label=y)

# Add a legend to the axis
plt.legend()

plt.show()

In [ ]:
	# change % netDebtToEBITDA	change % currentRatio

In [ ]:
plt.title('Percent')
plt.xlabel('Date')
plt.ylabel('Percent')


z = 'change % averageInventory'
e = 'change % debtToAssets'

# Plot close price and moving averages
plt.plot(df_with_changes[f'change % {ticker}'],lw=1, label=ticker)
plt.plot(df_with_changes[z],'g',lw=1, label=z)
plt.plot(df_with_changes[e],'r', lw=1, label=e)

# Add a legend to the axis
plt.legend()

plt.show()

In [ ]:
plt.title('Percent')
plt.xlabel('Date')
plt.ylabel('Percent')


z1 = 'change % cashPerShare'
e1 = 'change % debtToAssets'

# Plot close price and moving averages
plt.plot(df_with_changes[f'change % {ticker}'],lw=1, label=ticker)
plt.plot(df_with_changes[z1],'g',lw=1, label=z1)
plt.plot(df_with_changes[e1],'r', lw=1, label=e1)

# Add a legend to the axis
plt.legend()

plt.show()